## Compute the final result without ensembling

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import pandas as pd
import cv2
import matplotlib
import matplotlib.pyplot as plt
from skimage.transform import resize
from config import Config
import utils
import train
import validation
import model as modellib
import visualize
from model import log
from tqdm import tqdm

def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax
%matplotlib inline
# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")


X_test, sizes_test, test_ids = utils.load_test_data()

log_path = "logs/nuclei20180325T2205/mask_rcnn_nuclei_{:04d}.h5"

class InferenceConfig(train.NuConfig2):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MAX_INSTANCES = 200
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 1024
    IMAGE_PADDING = True

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)
        
        
model_path = log_path.format(218)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

new_test_ids = []
rles = []
for n, id_ in tqdm(enumerate(test_ids)):
    test_image = X_test[n]
    image = test_image
    results = model.detect([image], verbose=0)

    r = results[0]
    num_instances = r['masks'].shape[2]

    print(num_instances)
    try:
        rle = list(utils.prob_to_rles(r['masks'])) if len(r["scores"])>0 else [[]]

        rles.extend(rle)
        new_test_ids.extend([id_] * len(rle))
    
        lab_img = utils.masks2labels(r['masks'])
        print(lab_img.shape)
        masks = np.zeros_like(r['masks'])
        for i in range(1, lab_img.max() + 1):
            masks[:,:,i-1] = np.where(lab_img == i, 1, 0)

        #visualize.display_instances(image, r['rois'], masks, r['class_ids'], 
        #                        ['BG','N'], r['scores'], ax=get_ax())
    except ValueError: 
        print("ValueError")
        pass
    
# Create submission DataFrame
sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv('dsbowl2018-1.csv', index=False)

print('Finished')

## Compute the final result with ensembling ROIs

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import pandas as pd
import cv2
import matplotlib
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage import exposure
from skimage import img_as_ubyte

from config import Config
import utils
import train
import validation
import model as modellib
import visualize
from model import log
from tqdm import tqdm

def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax
%matplotlib inline
# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")


X_test, sizes_test, test_ids = utils.load_test_data()

log_path = "logs/nuclei20180325T2205/mask_rcnn_nuclei_{:04d}.h5"

class InferenceConfig(train.NuConfig2):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MAX_INSTANCES = 200
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 1024
    IMAGE_PADDING = True
    INPUT_ROIS_SIZE = 30000

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference_rois", 
                          config=inference_config,
                          model_dir=MODEL_DIR)
model2 = modellib.MaskRCNN(mode="inference_given_rois", 
                          config=inference_config,
                          model_dir=MODEL_DIR)
        
model_path = log_path.format(218) # which is the best according to validation
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)
model2.load_weights(model_path, by_name=True)

new_test_ids = []
rles = []
for n, id_ in tqdm(enumerate(test_ids)):
    test_image = X_test[n]
    image0 = test_image
    image1 = np.fliplr(test_image)
    image2 = np.flipud(test_image)
    
    rois = []
    scores = []

    # Original image
    results = model.detect_rois([image0], verbose=1)
    r = results[0]
    rois = rois + list(r['rois'])
    scores = scores + list(r['scores'])
    
    # Left-right flipped image
    results = model.detect_rois([image1], verbose=1)
    r = results[0]
    for i in range(r['rois'].shape[0]):
        r['rois'][i,1], r['rois'][i,3] = image1.shape[1] - r['rois'][i,3], image1.shape[1] - r['rois'][i,1]
    rois = rois + list(r['rois'])
    scores = scores + list(r['scores'])
    
    # Up-down flipped image
    results = model.detect_rois([image2], verbose=1)
    r = results[0]
    for i in range(r['rois'].shape[0]):
        r['rois'][i,0], r['rois'][i,2] = image1.shape[0] - r['rois'][i,2], image1.shape[0] - r['rois'][i,0]
    rois = rois + list(r['rois'])
    scores = scores + list(r['scores'])
    
    
    print('Total ROIs = ' + str(len(rois)))
    
    #visualize.display_rpn_rois(image, r['rois'], r['scores'], threshold = 0.5, ax=get_ax())
    
    results = model2.detect_given_rois([image0], [scores], [rois], verbose=1)
    
    r = results[0]
    num_instances = r['masks'].shape[2]
    
    rle = list(utils.prob_to_rles(r['masks'])) if len(r["scores"])>0 else [[]]

    rles.extend(rle)
    new_test_ids.extend([id_] * len(rle))
        
# Create submission DataFrame
sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv('dsbowl2018-1.csv', index=False)

print('Finished')